# Introduction to DataBases in Python

1 <b> Intro to databases </b>, relational model, connecting to database, engines and connection strings, autoloading tables from database, viewing table details, intro to SQL, selecting data from table(raw SQL), selecting data from table with SQLAlchemy
<br>
2 <b>Applying Filtering</b>, Ordering and Grouping to Queries
filtering and targeting data, connecting to PostreSQL DB, filter data selected from table, ordering by single column, descending order, ordering bt multiple columns, counting, summing, grouping,counting distinct data,use pandas and matplotlib to visialize our data, SQLAlchemy ResultsProxy and Pandas DataFrame, From SQLAlchemu results to a Graph
<br>
3 <b>ADVANCED SQLAlchemy</b> querries
calculing values  in query, connecting to MySQL DataBase, calculating difference between two columns, determening percentage, SQL relationships, automatic joins, joins,hierarchical tables, leveraging functions and Group_bys
<br>
4 <b>Creating and Manipulating your own database</b>
creating databases and tables, creating tables with SQLAlchemy, Inserting data to table, inserting a single row with an insert() statement, inserting multiple records at once, Loading CSV into Table,
Updating data in dataBase, updating individual records, updating multiple records, correlated updates, removing data from database, deleting all records from table, deleting specific records, deleting table completely
<br>
5 <b>Putting it all together - Census Case Study</b> <br>
Setup the engine and MetaData, create the table to the database, populating the database,
reading the data from csv, load data from list into the table, example querries,
build a query to determine  the average age by population, determine percentage of population by gender and state, determine the difference by state from the 2000 and 2008 censuses
In separate file:
99_Python_Datacamp_13_intro_to_databases_casestudy_census.ipynb
All other exercises in:
99_Python_Datacamp_13_intro_to_databases_exercises_sqlalchemy.ipynb

###  SQLAlchemy vs SQL

<b>WHERE </b>

SQL: 
SELECT * FROM census WHERE sex= F

SQLAlchemy:
db.select([census]).where(census.columns.sex == 'F')


<b>IN </b>
SQL :
SELECT state, sex
FROM census
WHERE state IN (Texas, New York)
SQLAlchemy :
db.select([census.columns.state, census.columns.sex]).where(census.columns.state.in_(['Texas', 'New York']))

<b>AND, OR, NOT</b>
SQL :
SELECT * FROM census
WHERE state = 'California' AND NOT sex = 'M'
SQLAlchemy :
db.select([census]).where(db.and_(census.columns.state == 'California', census.columns.sex != 'M'))

<b>ODER BY</b>

SQL :
SELECT * FROM census
ORDER BY State DESC, pop2000
SQLAlchemy :
db.select([census]).order_by(db.desc(census.columns.state), census.columns.pop2000)

<b>Functions</b>
SQL :
SELECT SUM(pop2008)
FROM census
SQLAlchemy :
db.select([db.func.sum(census.columns.pop2008)])

<b>GROUP BY</b>
SQL :
SELECT SUM(pop2008) AS pop2008, sex
FROM census
SQLAlchemy :
db.select([db.func.sum(census.columns.pop2008).label('pop2008'), census.columns.sex]).group_by(census.columns.sex)

<b>distinct </b>
SQL :
SELECT DISTINCT state
FROM census
SQLAlchemy :
db.select([census.columns.state.distinct()])

case & cast

The case() expression accepts a list of conditions to match and the column to return if the condition matches, followed by an else_ if none of the conditions match.

cast() function to convert an expression to a particular type

example
import sqlalchemy as db
In [3]:
engine = db.create_engine('sqlite:///census.sqlite')
connection = engine.connect()
metadata = db.MetaData()
census = db.Table('census', metadata, autoload=True, autoload_with=engine)
In [6]:
female_pop = db.func.sum(db.case([(census.columns.sex == 'F', census.columns.pop2000)],else_=0))
In [7]:
total_pop = db.cast(db.func.sum(census.columns.pop2000), db.Float)
In [8]:
query = db.select([female_pop/total_pop * 100])
In [9]:
result = connection.execute(query).scalar()
print(result)
51.09467432293413

<b>joins</b>

If you have two tables that already have an established relationship, you can automatically use that relationship by just adding the columns we want from each table to the select statement.

select([census.columns.pop2008, state_fact.columns.abbreviation])
import sqlalchemy as db
import pandas as pd

engine = db.create_engine('sqlite:///census.sqlite')
connection = engine.connect()
metadata = db.MetaData()

census = db.Table('census', metadata, autoload=True, autoload_with=engine)
state_fact = db.Table('state_fact', metadata, autoload=True, autoload_with=engine)

<b>Automatic Join</b>
query = db.select([census.columns.pop2008, state_fact.columns.abbreviation])
result = connection.execute(query).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(5)

<b>Manual Join</b>
query = db.select([census, state_fact])
query = query.select_from(census.join(state_fact, census.columns.state == state_fact.columns.name))
results = connection.execute(query).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(5)


## Creating and Inserting data into Tables


In [21]:
import sqlalchemy as db
import pandas as pd

#creating database and table
engine = db.create_engine('sqlite:///test.sqlite') #Create test.sqlite automatically
connection = engine.connect()
metadata = db.MetaData()

emp = db.Table('emp', metadata,
              db.Column('Id', db.Integer()),
              db.Column('name', db.String(255), nullable=False),
              db.Column('salary', db.Float(), default=100.0),
              db.Column('active', db.Boolean(), default=True)
              )

metadata.create_all(engine) #Creates the table

In [22]:
#inserting data
#Inserting record one by one
query = db.insert(emp).values(Id=1, name='naveen', salary=60000.00, active=True) 
ResultProxy = connection.execute(query)

In [23]:
#inserting many records at ones
query = db.insert(emp) 
values_list = [{'Id':'2', 'name':'ram', 'salary':80000, 'active':False},
               {'Id':'3', 'name':'ramesh', 'salary':70000, 'active':True}]
ResultProxy = connection.execute(query,values_list)

In [24]:
results = connection.execute(db.select([emp])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(4)

,Id,name,salary,active
0,1,naveen,60000.0,True
1,2,ram,80000.0,False
2,3,ramesh,70000.0,True


### Updating data in DataBases
db.update(table_name).values(attribute = new_value).where(condition)

In [25]:
engine = db.create_engine('sqlite:///test.sqlite')
metadata = db.MetaData()
connection = engine.connect()
emp = db.Table('emp', metadata, autoload=True, autoload_with=engine)

results = connection.execute(db.select([emp])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(4)

,Id,name,salary,active
0,1,naveen,60000.0,True
1,2,ram,80000.0,False
2,3,ramesh,70000.0,True


In [26]:
# Build a statement to update the salary to 100000
query = db.update(emp).values(salary = 100000)
query = query.where(emp.columns.Id == 1)
results = connection.execute(query)

results = connection.execute(db.select([emp])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(4)

,Id,name,salary,active
0,1,naveen,100000.0,True
1,2,ram,80000.0,False
2,3,ramesh,70000.0,True


### delete table
db.delete(table_name).where(condition)

In [27]:
engine = db.create_engine('sqlite:///test.sqlite')
metadata = db.MetaData()
connection = engine.connect()
emp = db.Table('emp', metadata, autoload=True, autoload_with=engine)

results = connection.execute(db.select([emp])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(4)

,Id,name,salary,active
0,1,naveen,100000.0,True
1,2,ram,80000.0,False
2,3,ramesh,70000.0,True


In [28]:
# Build a statement to delete where salary < 100000
query = db.delete(emp)
query = query.where(emp.columns.salary < 100000)
results = connection.execute(query)

results = connection.execute(db.select([emp])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(4)


,Id,name,salary,active
0,1,naveen,100000.0,True


In [ ]:
#Convert to DATA FRAME
query = db.select([census.columns.state.distinct()])
result = connection.execute(query).fetchall()
result[:3]

df = pd.DataFrame(result)
df.columns = result[0].keys()
df.head(2)

In [ ]:
### FROM DATACAMP

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://census_nyc.sqlite')
connection = engine.connect()

In [ ]:
#what is in your database
print(engine.table_names())

In [ ]:
#reflection reads database and builds SQLAlchemy Table object
from sqlalchemy import MetaData, Table
metadata = MetaData()
census = Table('census')
print(repr(census))

In [ ]:
#Basic SQL querying
SELECT column_name FROM table_name
SELECT pop2008 FROM people
SELECT * FROM People

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://census_nyc.sqlite')
connection = engine.connect()
stmt = 'SELECT * FROM people'
result_proxy = connection.execute(stmt)
result=result_proxy.fetchall()

In [ ]:
#handling resultSets
first_row = results[0]
print(first_row)

print(first_row.keys())

print(first_row.state)

In [ ]:

metadata = MetaData()
census = Table('census', metadata, autoload = True, autoload_with = engine)
census = Table('census', metadata, autoload = True, autoload_with = engine)
stmt = select([census])
results = connection.execute(stmt).fetchall()

print(stmt)

### Filtering and Targeting Data


Where Clauses


In [ ]:
stmt = select([census])
stmt = stmt.where(census.columns.state == 'California')

results = connection.execute(stmt).fetchall()

for result in results:
    print(result.state, result.age)

In [ ]:
stmt = select([census])
stmt = stmt.where(census.columns.state.startswith('New'))

for result in connection.execute(stmt):
    print(result.state, result,pop2000)

In [ ]:
from sqlalchemy import or_
stmt = select([census])
stmt = stmt.where(
    or_(census.columns.state == 'California',
       census.columns.state == 'New York'))

for result in connection.execute(stmt):
    print(result.state, result.sex)

### Ordering Query Results

Order by clauses allows us to control the order in which records are returned in the query results
Available as a method on statements order_by()

#### Order by Ascending

In [ ]:
print(results[:10])

stmt = select([census.columns.state])
stmt = stmt.order_by(census.columns.state)

results = connection.execute(stmt).fetchall()

print(results[:10])

#### Order by Descending
wrap the column with desc() in order_by() clause

#### Order by Multiple
just separate multiple columns with a comma
orders completely by the first column
then if there are duplicates in the first column, orders by the second column
repeat until all columns are ordered

In [ ]:
#order by multiple

stmt = select([census.columns.state, census.columns.sex])

stmt = stmt.order_by(census.columns.state, census.columns.sex)

results = connection.execute(stmt).first()
print(results)

#### Counting Summing and Grouping data
SQL FUnctions
COunt,Sum, from sqlalchemy import func
more efficient than processing in Python
aggregate data

In [ ]:
#sum example
from sqlalchemy import func
stmt = select([func.sum(census.columns.pop2008)])
results = connection.execute(stmt).scalar()
print(results)

In [ ]:
#groupby example  single
stmt = select([census.columns.sex, func.sum(census.columns.pop2008)])
stmt = stmt.group_by(census.columns.sex)
results = connection.execute(stmt).fetchall()

print(results)

In [ ]:
#groupby example  single
stmt = select([census.columns.sex,
               census.columns.age,
               func.sum(census.columns.pop2008)])
stmt = stmt.group_by(census.columns.sex,
                    census.columns.age)
results = connection.execute(stmt).fetchall()

print(results)

#### label()
SQLAlchemu auto generates 'column names' for functions in the ResultSet
The column names are often func_# such as count_1
replace them with the label() method

In [ ]:
#using label()
print(results[0].keys())

#result will look like
['sex', u'sum_1']


stmt = select([census.columns.sex,
              func.sum(census.columns.pop2008).label(
              'pop2008_sum')])

stmt=stmt.group_by(census.columns.sex)
results = connection.execute(stmt).fetchall()

print(results[0]).keys())
#result will look like
['sex', ' pop2008_sum']


## SQLAlchemy and Pandas for Visualization

In [ ]:
#graphing example
import matplotlib.pyplot as plt
df[10:20].plot.barh()  #bar graphs
plt.show()

##   ADVANCED SQLALCHEMY queries
Calculating values in a quary
math operators: addition, subtraction, multiplication, division, modulus %

In [ ]:
#calculating difference

stmt= select([census.columns.age,
             (census.columns.pop2008-
             census.columns.pop2000).label('pop_change')])

stmt = stmt.groupby(census.columns.age)
stmt = stmt.order_by(desc('pop_change'))
stmt = stmt.limit(5)
results = conection.execute(stmt).fetchall

print(results)

In [ ]:
#case statement example
from sqlalchemy import case
stmt = select([func.sum(
                  case([
                      (census.columns.state == 'New York',
                      census.columns.pop2008)
                  ], else_=0))])
results = connection.execute(stmt).fetchall()
print(results)

### Cast statement
converts data to another type
useful for converting
integers to floats for division
strings to dates and times

accepts a column or expression and the target Type


#### Percentage example


https://github.com/wblakecannon/DataCamp/blob/master/13-introduction-to-databases-in-python/03-advanced-sqlalchemy-queries/ch3_slides.pdf

In [ ]:
#percentage example
from sqlalchemy import case,cast,Float

stmt = select([
    (func.sum(
    case([
        (census.columns.state == 'New York',
        census.columns.pop2008)
    ], else_=0)) /
     cast(func.sum(census.columns.pop2008),
         Float)*100).label('ny_percent')])
results = connection.execute(stmt).fetchall()
print(results)

### SQL Relationships
allow us to avoid duplicate data
make it easy to change things in one place
useful to break out information from table we dont need very often

### Automatic joins

In [ ]:
stmt = select([census.columns.pop2008,
              state_fact.columns.abbreviation])
results = connection.execute(stmt).fetchall()
print(results)

#### join
accepts a Table and an optional expression that explains how the two tables are related
The expression is not needed if the relationship is predefined and available via reflection
COmes immediately after the select() clause and priorto any where(), order_by or group_by() clauses


#### select_from
used to replace the default, derived FROM clause with a join, wraps the join() clause

In [ ]:
# select_from example
stmt = select([func.sum(census.columns.pop2000)])

stmt = stmt.select_from(census.join(state_fact))
stmt = stmt.where(state_fact.columns.circuit_court == '10')
result = connection.execute(stmt).scalar()
print(result)

#### joining tables without predefined relationship

Join accepts a Table and an optional expression that explains how the two tables are related
Will only join on data that match between the two columns
Avoid joining on columns of different types

In [ ]:
#select from example
stmt = select([func.sum(census.columns.pop2000)])

stmt = stmt.select_from(
        census.join(state_fact,census.columns.state
                   == state_fact.columns.name))

stmt = stmt.where(
        state_fact.columns.census_division_name ==
        'East South Central')

result = connection.execute(stmt).scalar()
print(result)

### Working with Hierarchical Tables
#### Hierarchical Tables
Contain a relationship with themselves

alias() - requires a way to view the table via multiple names
creates a unique reference that we can use

In [ ]:
#querying hierarchical data
managers = employees.alias()

stmt = select(
[managers.columns.name.label('manager'),
employees.columns.name.label('employee')])

stmt = stmt.select_from(employees.join(
managers,managers.columns.id == 
employees.columns.manager))

stmt = stmt.order_by(managers.columns.name)

print(connection.execute(stmt).fetchall())

#### group_by and func
its important to target group_by() at the tight alias

if you dont find yourself using both the alias and the table name for a query, dont create the alias at all


#### querying hierarchical data


In [ ]:
managers = employees.alias()
stmt = select([managers.columns.name,
              func.sum(employees.columns.sal)])

stmt = stmt.select_from(employees.join(managers,managers.columns.id ==
                                      employees.columns.manager))

stmt = stmt.group_by(managers.columns.name)

print(connection.execute(stmt).fetchall())

### Handling large result sets
fetchmany() lets us specify how many rows we want to act upon
we can loop over fetchmany()
it returns an empty list when there are no more records
we have to close the ResultProxy afterwards

In [ ]:
#fetching many rows
while more results:
    partial_results = results_proxy.fetchmany(50)
    if partial_results == []:
        more_results = False
        
    for row in partial_results:
        state_count[row.state] += 1
results_proxy.close()

## Creating and Manipulating your own database

### Creating databases
varies by the database type
databases like PostgreSQL and MySQL have command line tools to initialize the database

with SQLite the create_engine() statement will create the database and file if they do not exist already


#### Examle of building a table

In [5]:
from sqlalchemy import (Table,Column,String,Integer,Boolean)
from sqlalchemy import MetaData, Table, create_engine, select, func, desc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

metadata = MetaData()

engine = create_engine("sqlite:///:memory:", echo=True)

data = Table('data', metadata,
                 Column('name', String(255)),
                 Column('id', Integer()),
                 Column('salary', Integer()), #changed from decimal to integer
                 Column('active', Boolean()))
# Use the metadata to create the table
metadata.create_all(engine)

engine.table_names()

2019-06-14 10:37:43,186 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-06-14 10:37:43,193 INFO sqlalchemy.engine.base.Engine ()
2019-06-14 10:37:43,194 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-06-14 10:37:43,195 INFO sqlalchemy.engine.base.Engine ()
2019-06-14 10:37:43,196 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("employees")
2019-06-14 10:37:43,198 INFO sqlalchemy.engine.base.Engine ()
2019-06-14 10:37:43,200 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE employees (
	id INTEGER, 
	name VARCHAR(255), 
	salary INTEGER, 
	active BOOLEAN, 
	CHECK (active IN (0, 1))
)


2019-06-14 10:37:43,201 INFO sqlalchemy.engine.base.Engine ()
2019-06-14 10:37:43,202 INFO sqlalchemy.engine.base.Engine COMMIT
2019-06-14 10:37:43,203 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-06-14 10:37:43,203 INFO sqlalchemy.engine.ba

['employees']

create_all() - creates the tables in the actual database by using the create_all() method on the MetaData instance

#### Additional column options
unique forces all values for the data in a column to be unique
nullable determines if a column can be empty in a row
default sets a default values if one isnt supplied

#### Building a table with additional options


In [6]:
employees2 = Table('employees2', metadata,
                 Column('id', Integer()),
                 Column('name', String(255), unique = True, nullable = False),  #additional options
                 Column('salary', Integer(), default = 100), #additional option
                 Column('active', Boolean(), default = True))  #additional option
employees2.constraints

{CheckConstraint(<sqlalchemy.sql.elements.BinaryExpression object at 0x000001CAB9A32780>, name='_unnamed_', table=Table('employees2', MetaData(bind=None), Column('id', Integer(), table=<employees2>), Column('name', String(length=255), table=<employees2>, nullable=False), Column('salary', Integer(), table=<employees2>, default=ColumnDefault(100)), Column('active', Boolean(), table=<employees2>, default=ColumnDefault(True)), schema=None), _create_rule=<sqlalchemy.util.langhelpers.portable_instancemethod object at 0x000001CAB9ABADC8>, _type_bound=True),
 PrimaryKeyConstraint(),
 UniqueConstraint(Column('name', String(length=255), table=<employees2>, nullable=False))}

### Inserting data into table

done with insert() statement
Insert() takes the table we are loading data into as the argument
we add all the values we want to insert in with the values clause as column = value pairs
doesnt return any rows, so no need for a fetch method

#### inserting one row example

In [7]:
from sqlalchemy import insert

connection =engine.connect()
stmt = insert(employees).values(id=1, name='Jason', salary=1, active=True)
result_proxy = connection.execute(stmt)
print(result_proxy.rowcount)

2019-06-14 10:52:00,375 INFO sqlalchemy.engine.base.Engine INSERT INTO employees (id, name, salary, active) VALUES (?, ?, ?, ?)
2019-06-14 10:52:00,377 INFO sqlalchemy.engine.base.Engine (1, 'Jason', 1, 1)
2019-06-14 10:52:00,384 INFO sqlalchemy.engine.base.Engine COMMIT
1


#### inserting multiple rows
build an insert statement without any values
build a list of dictionaries that represent all the values clauses for the rows you want to insert
pass both the stmt and the values list to the execute method on connection

In [8]:
stmt = insert(employees)

values_list = [
    {'id':2, 'name': 'Rebecca', 'salary':2, 'active': True},
    {'id':3, 'name': 'Bob', 'salary':0, 'active': False}
]
result_proxy = connection.execute(stmt, values_list)
print(result_proxy.rowcount)

2019-06-14 10:56:17,071 INFO sqlalchemy.engine.base.Engine INSERT INTO employees (id, name, salary, active) VALUES (?, ?, ?, ?)
2019-06-14 10:56:17,071 INFO sqlalchemy.engine.base.Engine ((2, 'Rebecca', 2, 1), (3, 'Bob', 0, 0))
2019-06-14 10:56:17,073 INFO sqlalchemy.engine.base.Engine COMMIT
2


### Updating data in a table
done with update statement
similar to the insert statement but includes a where clause to determine what record will be updated
we add all the values we want to update with the values clause as column=value pairs

#### Updating one row

In [9]:
from sqlalchemy import update
stmt = update(employees)
stmt = stmt.where(employees.columns.id == 3)
stmt = stmt.values(active = True)
result_proxy = connection.execute(stmt)
print(result_proxy.rowcount)

2019-06-14 11:00:00,662 INFO sqlalchemy.engine.base.Engine UPDATE employees SET active=? WHERE employees.id = ?
2019-06-14 11:00:00,664 INFO sqlalchemy.engine.base.Engine (1, 3)
2019-06-14 11:00:00,665 INFO sqlalchemy.engine.base.Engine COMMIT
1


#### Updating multiple rows / inserting multiple rows
build a where clause that will select all the record you want to update

In [10]:
stmt = update(employees)
stmt = stmt.where(employees.columns.active == True)
stmt = stmt.values(active=False, salary=0)
result_proxy=connection.execute(stmt)
print(result_proxy.rowcount)

2019-06-14 11:03:03,271 INFO sqlalchemy.engine.base.Engine UPDATE employees SET salary=?, active=? WHERE employees.active = 1
2019-06-14 11:03:03,272 INFO sqlalchemy.engine.base.Engine (0, 0)
2019-06-14 11:03:03,273 INFO sqlalchemy.engine.base.Engine COMMIT
3


### Correlated Updates
uses a select() statement to find the value for the column we are updating
commonly used to update records to a maximum value or change a string to match an abbreviation from another table

In [11]:
new_salary = select([employees.columns.salary])
new_salary = new_salary.order_by(desc(employees.columns.salary))

new_salary = new_salary.limit(1)
stmt = update(employees)
stmt = stmt.values(salary = new_salary)
result_proxy = connection.execute(stmt)
print(result_proxy.rowcount)

2019-06-14 12:54:30,298 INFO sqlalchemy.engine.base.Engine UPDATE employees SET salary=(SELECT employees.salary 
FROM employees ORDER BY employees.salary DESC
 LIMIT ? OFFSET ?)
2019-06-14 12:54:30,299 INFO sqlalchemy.engine.base.Engine (1, 0)
2019-06-14 12:54:30,300 INFO sqlalchemy.engine.base.Engine COMMIT
3


In [12]:
print(result_proxy)

### Deleting Data from a DataBase
done with the delete() statement
delete() takes the table we are loading data into as the argument
A where() clause is used to choose which rows to delete

HARD to UNDO!!!!

#### Deleting all data from table

In [ ]:
from sqlalchemy import delete
stmt = select([
    func.count(extra_employees.columns.id)])
connection.execute(stmt).scalar()

In [ ]:
delete_stmt = delete(extra_employees)
result_proxy = connection.execute(delete_stmt)
result_proxy.rowcount

#### deleting specific rows
build a where clause that will select all the records you want to delete

In [ ]:
stmt = delete(employees).where(employees.columns.id == 3)
result_proxy = connection.execute(stmt)
result_proxy.rowcount

#### dropping  a table completely
uses the drop method on table
accepts the engine as an argument so it knows where to remove the table from
won't remove it from metadata until the python process in restarted



In [ ]:
extra_employees.drop(engine)
print(extra_employees.exists(engine))
False

#### dropping all the tables
uses the drop_all() method on MetaData

In [ ]:
metadata.drop_all(engine)
engine.table_names()

## Case study, census database
preparing SQLAlchemy and the DataBase
Loading Data into the Database
Solving data science problems with queries

In separate file
99_Python_Datacamp_13_intro_to_databases_casestudy_census.ipynb